In [178]:
import tensorflow as tf
import pandas as pd
import numpy as np
import pickle
from tensorflow.keras.models import load_model

In [179]:
## Load trained model , scaler pickle , OHE

model = load_model('model.h5')
scaler = pickle.load(open('scaler.pkl','rb'))
ohe = pickle.load(open('ohe_encoder_geo.pkl','rb'))
lable = pickle.load(open('lable_encoder_gender.pkl','rb'))

In [180]:
# Example data

input_data = {
    'CreditScore' : 600,
    'Gender' : 'Male',
    'Geography' : 'France',
    'Age' : 40,
    'Tenure' : 3,
    'Balance' : 60000,
    'NumOfProducts' : 2,
    'HasCrCard' : 1,
    'IsActiveMember' : 1,
    'EstimatedSalary' : 50000
}

In [181]:
input_data_df = pd.DataFrame(input_data,index=[0])
input_data_df

,CreditScore,Gender,Geography,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,Male,France,40,3,60000,2,1,1,50000


In [182]:
## apply OHE to Geography

geo_encoded = ohe.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns = ohe.get_feature_names_out())
geo_encoded_df

e:\AI ML\Projects\DL projects\ANN classification\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [183]:
## apply lable encoder to Gender

input_data_df['Gender'] = lable.transform([input_data_df['Gender']])


e:\AI ML\Projects\DL projects\ANN classification\venv\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [184]:
input_data_df

,CreditScore,Gender,Geography,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,1,France,40,3,60000,2,1,1,50000


In [185]:
input_data_df = pd.concat([input_data_df.drop('Geography',axis=1),geo_encoded_df],axis=1)


In [ ]:
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


: 

In [186]:
## scaling data

input_scaled = scaler.transform(input_data_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [187]:
## prediction

prediction = model.predict(input_scaled)
prediction_prob = prediction[0][0]
prediction_prob

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step


np.float32(0.04931594)

In [188]:
if prediction_prob > 0.5:
    print('Churn')
else:
    print('Not Churn')

Not Churn
